In [ ]:
DATASET_NAME = 'vk'
LABELING_OPTION = None
MAX_LEN = 60

In [ ]:
# YOU CAN COMMENT THIS IF YOU DO NOT USE COLAB
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip -q install -U sentence-transformers &>/dev/null

In [ ]:
import json
import re
import os
import requests
import matplotlib.pyplot as plt
import pickle
import string
import functools
import gc
from tqdm.notebook import tqdm
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
SEED = 1337
np.random.seed(SEED)

import random
random.seed(SEED)
# https://www.sbert.net/docs/pretrained_models.html
# from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split

In [ ]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None
def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


if __name__ == "__main__":
    if not os.path.exists('data'):
        os.mkdir('data')

    json_id = '1iCcIlPg3L1J-K0hz4OBnofJ4bq75RK9p'
    dest  = 'data/y3_parsed.v3.tags_sep.json'
    print('downloading...')
    download_file_from_google_drive(json_id, dest)
    print('downloaded!')
    

downloading...
downloaded!


Мета - словарь с вложенными словарями на каждый внешний ключ

In [ ]:
# download
with open(dest) as fh:
    posts = json.load(fh)

https://stackoverflow.com/questions/63483913/classification-for-a-case-with-multiple-labels-per-data-point

# Tags saving

In [ ]:
# Count the number of occurences of the parsed tags
cntr = Counter([x for y in posts['tags_parsed'].values() for x in y])
how_many = 15
for row in cntr.most_common(how_many):
    print(row)

cntr = list(cntr.items())
missing_tag = 'null'
cntr.append((missing_tag, 0))
low_labels = []  # теги частотность которых меньше `low_limit` - удаляем
low_limit = 0


# 5000 for tags, 20000 for classes
high_limit = 20000
high_count = {}  # теги которые сохраним

for k, v in cntr:
    # FIXED: before was `v <= low_limit`
    if v < low_limit:
        low_labels.append(k)
    else:
        high_count[k] = 0

('Новость', 21454)
('#лучшедома', 9775)
('Мероприятие', 9162)
('Вопрос', 7766)
('Благоустройство', 6823)
('Настроение', 5636)
('Дети', 4389)
('Предупреждение', 4214)
('Безобразие', 4157)
('Здоровье', 3727)
('Услуги', 3421)
('Водителю', 3029)
('Происшествие', 2997)
('Совет', 2606)
('Спорт', 2541)


In [ ]:
list(Counter([x for y in posts['tags_parsed'].values() for x in y]).keys()).__len__()

47

In [ ]:
raw_sents = [sent for sent in list(posts['text'].values())]
raw_tags = [tag for tag in list(posts['tags_parsed'].values())]

In [ ]:
def remove_duplicates(raw_sents, raw_tags):
    d = {}
    for s, t in zip(raw_sents, raw_tags):
        if s not in d:
            d[s] = t
    unique_raw_sents = list(d.keys())
    unique_raw_tags = list(d.values())
    return unique_raw_sents, unique_raw_tags

In [ ]:
raw_sents, raw_tags = remove_duplicates(raw_sents, raw_tags)

In [ ]:
i = 3

for i in range(i - 2, i + 2):
    print(raw_sents[i])
    print(raw_tags[i])
    print()

Ланское шоссе 16/2 не моют лестницы. Очень грязно. Где профилактика, о которой говорят по телевидению?
['#лучшедома']

Кемеровчанин заплатил двойную цену за эвакуацию на родину с Украины<br/><br/>Кемеровчанину Антону Галееву, из-за семейных обстоятельств оказавшемуся в Украине в разгар пандемии коронавируса, пришлось дважды заплатить за возвращение домой.<br/><br/><a class="link" href="https://vse42.ru/news/31158171" rel="nofollow noreferrer noopener" target="_blank">https://vse42.ru/news/31158171</a>
['#лучшедома']

Смотрите, что происходит на площади Куйбышева! Новым годом уже пахнет вовсю! Соседи, а у вас новогоднее настроение есть? Или может у вас рядом с домом уже тоже украшения поставили новогодние?<br/>Фото: Светлана Маковеева
['Новость', 'Настроение', 'Благоустройство']

Сегодня увидела в 2ГИС, что можно искать лекарства на карте.<br/>Очень помогает понимать где что есть и сколько стоит. Особенно в условиях когда хочется с первого раза найти аптеке то что нужно и по хорошей цен

# Data preprocessing

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def de_emojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ', text)


extras = ['а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с',
          'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ь', 'ъ', 'э', 'ю', 'я',
          'этот', 'эти', 'эта', 'это', 'также', 'так', 'тот', 'потом', 'же']


def process_text(text,
                 tags,
                 text_part=1,
                 min_word_size=4,
                 min_sent_size=5,
                 max_sent_size=None,
                 extra_stop=None):
    # HOW TO REMOVE UNCLOSED HTML TAGS ---> https://datadependence.com/2016/03/find-unclosed-tags-using-stacks/
    html_cleaner = re.compile('<.*?>')
    cyrillic = re.compile(r'[^а-яА-Я0-9 ]')
    esc_punctuation = re.compile('[%s]' % re.escape(string.punctuation))  # let's try not to remove punctuation
    extra_spaces = re.compile(r'\s{2,}')
    stop_list = stopwords.words('russian')

    if extra_stop:
        stop_list.extend(extra_stop)
    
    n_sents = int(len(text) * text_part)
    random_idxs = np.random.randint(0, len(text), n_sents)
    text = np.asarray(text)[random_idxs]
    tags = np.asarray(tags)[random_idxs]
    for i in tqdm(range(n_sents)):
        # html removal
        text[i] = html_cleaner.sub(' ', text[i]) # there's NO NoneType error
        # punctuation and numbers removal
        # text[i] = esc_punctuation.sub(' ', text[i])
        
        # leaving only cyrillic words
        text[i] = cyrillic.sub(' ', text[i])

        # remove emojis
        text[i] = de_emojify(text[i])

        # extra spacing
        text[i] = extra_spaces.sub(' ', text[i])
       
        # LOWERING REMOVED
        # stopwords removal + lowercasing + limitations to the number of words in a sentence ([:max_sent_size])
        if max_sent_size:
            text[i] = ' '.join([word for word in text[i].split()[:max_sent_size] \
                                            if len(word) >= min_word_size])# and word not in stop_list])
        # max sent is allowed
        else:
            text[i] = ' '.join([word for word in text[i].split() \
                                            if len(word) >= min_word_size])# and word not in stop_list])
    text_to_save = []
    tags_to_save = []
    for sent, tag in zip(text, tags):
        if len(sent) >= min_sent_size:
            text_to_save.append(sent)
            tags_to_save.append(tag)
    return text_to_save, tags_to_save

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus, tags = process_text(raw_sents,
                            raw_tags,
                            text_part=1,
                            min_word_size=1,
                            max_sent_size=MAX_LEN)

In [ ]:
print(len(corpus), len(tags), len(set(corpus)))

59632 59632 37593


In [ ]:
# еще раз удалим повторные предложения
# после препроцессинга некоторые предложения стали одинаковыми
corpus, tags = remove_duplicates(corpus, tags)

In [ ]:
print(len(corpus), len(tags), len(set(corpus)))

37593 37593 37593


In [ ]:
def form_1d_lists(sents, raw_tags, missing_tag=missing_tag, include_missing_tags=True):
    sents_to_save = []
    random_tags = []
    all_tags = []

    for s, ts in zip(sents, tags):
        # убираем случаи, когда `None in ts`
        ts = list(filter(lambda x: x, ts))
        # тега нет, i.e. []

        if not ts and include_missing_tags:
            t = missing_tag
            ts = [missing_tag]  # для проверки тимура
        # только один тег
        elif len(ts) == 1:
            t = ts[0]
        # тегов > 1
        elif len(ts) > 1:
            while len(ts) > 1:
                # случай нескольких тегов
                t = np.random.choice(ts, 1)[0]
                ts.remove(t)
                if high_count[t] < high_limit:
                    break
        else:
            continue

        if high_count[t] < high_limit:
            high_count[t] += 1
            sents_to_save.append(s)
            random_tags.append(t)
            all_tags.append(ts)

    
    return sents, random_tags, all_tags

```python
1.
    include_missing_tags=False
    используется для базовой классификации тегов / классов, то есть без использования кластеризации
2.
    include_missing_tags=True
    используется для классификации с предварительной перетасовкой тегов кластеризацией
```

In [ ]:
sents, tags, all_tags = form_1d_lists(corpus, tags, include_missing_tags=True)

len(tags), len(sents)

(37593, 37593)

In [ ]:
np.unique(np.asarray(tags)).size

47

### Загрузка новой разметки

In [ ]:
if DATASET_NAME == 'vk':
    DATASET_PATH = f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_new_{DATASET_NAME}.npy'
else:
    DATASET_PATH = f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_new_{DATASET_NAME}_{LABELING_OPTION}.npy'

dataset = np.load(file=DATASET_PATH,
                  allow_pickle=True)

In [ ]:
import textwrap

init = np.zeros((len(corpus), 2), dtype='object')
for i, (sent, tag) in enumerate(zip(corpus, tags)):
    init[i] = [sent, tag]

idxs = np.where(init[:, 1] != 'null')[0]
init = init[idxs]







In [ ]:
i = int(input()) # от 0 до 37592

print(f'Предложение: ', end='\n')
print("\n".join(textwrap.wrap(init[i, 0], 100)))
print()

print(f'лейбл до: {init[i, 1]}\n')
print(f'лейбл после: {dataset[idxs][i, 1]}')

847
Предложение: 
Фестиваль коллекционных автомобилей РетроФест пройдет в парке Сокольники Любители всего старинного
смогут посетить выставку ретро автомобилей которая будет проходить в течение двух дней в парке
Сокольники Время работы фестиваля 26 и 27 августа с 10 00 до 21 00 Посетить его можно будет
абсолютно бесплатно

лейбл до: Водителю

лейбл после: Мероприятие


## Train / test сплит

1. находим предложения с ненулевым тегом (классом) $\Rightarrow$ df
2. df разбиваем на трейн сплит $\Rightarrow$ train, test
3. train будет участвовать в выборе самого часто встречающегося тега для кластера

## Save BASE labels (no clustering)

In [ ]:
sent_tag_no_clust = np.zeros((len(corpus), 2), dtype='object')
for i in range(len(sent_tag_no_clust)):
        sent_tag_no_clust[i] = [corpus[i], tags[i]]

# if DATASET_NAME == 'vk':
#     np.save(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_old_{DATASET_NAME}', sent_tag_no_clust)
# else:
#     np.save(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_old_{DATASET_NAME}_{LABELING_OPTION}', sent_tag_no_clust)

In [ ]:
(sent_tag_no_clust[:, 1] != 'null').sum()

37593

# Activate CUDA

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
embedder = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
embedder = embedder.to(device)

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

# Base class

In [ ]:
import time

class Base(Dataset):
    def __init__(self, corpus=corpus, tags=tags, max_len=42):
        self.corpus = corpus
        self.tags = tags
        self.max_len = max_len
        self.data_loader = None
        self.corpus_embeddings = None

    def __len__(self):
        return len(self.corpus)

    def tokenize(self):
        self.encoded_input = tokenizer(self.corpus,
                                    return_token_type_ids=False,
                                    pad_to_max_length=True,
                                    return_attention_mask=True,
                                    return_tensors='pt',
                                    truncation=True,
                                    max_length=self.max_len)
        return self

        

    def __getitem__(self, item):
        return {
            'input_ids': self.encoded_input['input_ids'][item].flatten(),  # important to flatten [1, max_length] --> [max_length]
            'attention_mask': self.encoded_input['attention_mask'][item].flatten()
        }

    def create_data_loader(self, batch_size=16):
        self.data_loader = DataLoader(
                                    self,
                                    batch_size=batch_size,
                                    num_workers=4
                                    )
        return self

    def mean_pooling(self, model_output, attention_mask):
        torch.cuda.empty_cache()
        gc.collect()
        # float32 -> float16
        attention_mask = attention_mask.to(device)  # if cpu is set for model_output
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).half()  # .float()
        assert(input_mask_expanded.dtype == torch.float16)
        cur = model_output * input_mask_expanded
        model_output = None
        torch.cuda.empty_cache()
        gc.collect()
        sum_embeddings = torch.sum(cur, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def make_embeddings(self, queries=None):
        model_output = None
        start = time.time()
        assert(self.data_loader is not None)
        with torch.no_grad():
            for batch in tqdm(self.data_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                batch_output = embedder(input_ids=input_ids,
                                        attention_mask=attention_mask)
                # float32 to float16
                for k in batch_output:
                    batch_output[k] = batch_output[k].half()
                # first batch
                if not model_output:
                    model_output = batch_output
                # concat with the previous result
                else:
                    model_output['last_hidden_state'] = torch.cat((model_output['last_hidden_state'],
                                                                batch_output['last_hidden_state']))
                    model_output['pooler_output'] = torch.cat((model_output['pooler_output'],
                                                            batch_output['pooler_output']))


        model_output = model_output[0]#.cpu() # First element of model_output contains all token embeddings
        del self.encoded_input['input_ids']

        torch.cuda.empty_cache()
        gc.collect()

        self.corpus_embeddings = self.mean_pooling(model_output, self.encoded_input['attention_mask']) ###########
        self.encoded_input = None
        model_output = None
        torch.cuda.empty_cache()
        gc.collect()

        self.corpus_embeddings = self.corpus_embeddings.cpu()

        finish = time.time()
        total = finish - start
        print(f'Time taken: {int(total // 60)} minutes and {int(total % 60)} seconds')
        
        return self

# Clustering

In [ ]:
# GET indices from train_test_split
# https://towardsdatascience.com/3-things-you-need-to-know-before-you-train-test-split-869dfabb7e50
def get_train_test(corpus, tags, test_size=0.2):
    df = np.zeros((len(corpus), 2), dtype='object')
    for i, (sent, tag) in enumerate(zip(corpus, tags)):
        df[i] = [sent, tag]

    # выбираем индексы предложений, для которых есть тег
    idxs = np.where(df[:, 1] != 'null')[0]  # [0] чтобы достать массив индексов
    X, y = df[:, 0], df[:, 1]
    train_indices, test_indices = train_test_split(idxs, test_size=0.2, shuffle=True, random_state=42)
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices] 
    return X_train, X_test, y_train, y_test, train_indices, test_indices

In [ ]:
class Clustering(Base):

    def cluster(self, model_class, min_n_clusters, max_n_clusters, **model_params):
        '''
        cluster sentences with self.best_k chosen by self.choose_best_k
        '''
        self.X_train, self.X_test, \
        self.y_train, self.y_test, \
        self.train_idxs, self.test_idxs = get_train_test(self.corpus, self.tags)
        arr = np.zeros((len(self.corpus), 2), dtype='object')
        for i, (sent, tag) in enumerate(zip(self.corpus, self.tags)):
            arr[i] = [sent, tag]
        self.model = self.choose_best_k(model_class,
                                   min_n_clusters,
                                   max_n_clusters,
                                   **model_params)
        print(f'Clustering with the best k = {self.best_k}')
        self.model.fit(self.corpus_embeddings)
        print('#### DONE ####')        
        # получение номеров кластеров для всех предложений
        self.cluster_labels = self.model.predict(self.corpus_embeddings)
        # сохраним каждый кластер предложений (с тегами) в отдельном списке
        self.clustered_sentences_with_tags = [[] for i in range(self.best_k)]
        # добавляем пару (предложение, исходный тег)
        for sentence_id, cluster_id in enumerate(self.cluster_labels):
            self.clustered_sentences_with_tags[cluster_id].append(arr[sentence_id])
    
        self.clustered_sentences_with_tags = np.array([np.array(self.clustered_sentences_with_tags[i]) \
                                                                for i in range(self.best_k)])
        return self

    def choose_best_k(self, model_class, min_n_clusters, max_n_clusters, **model_params):
        K = range(min_n_clusters, max_n_clusters + 1)
        models = []  # stores all K models
        for k in K:
            print(f'Running with k = {k}')
            models.append(model_class(k, **model_params).fit(self.corpus_embeddings))
            print(f"\tDone for k = {k}")
        start = time.time()

        cluster_labels = [m.predict(self.corpus_embeddings) for m in models]

        silhouette_avgs = [silhouette_score(X=self.corpus_embeddings, labels=cluster_label) \
                            for cluster_label in cluster_labels]
        sil_k_avg_dict = []
        for k, sil_avg in zip(list(K), silhouette_avgs):
            sil_k_avg_dict.append(
                {
                    'savg': sil_avg, 
                    'n_clusts': k
                })
        finish = time.time()
        total = finish - start
        print(f'Time taken: {int(total // 60)} minutes and {int(total % 60)} seconds')
        self.sil_avgs = sorted(sil_k_avg_dict, key=lambda elem: elem['savg'], reverse=True)
        self.best_k = self.sil_avgs[0]['n_clusts']
        return model_class(self.best_k, **model_params)

    def create_dominant_tags_in_clusters(self):
        '''
        возвращает словарь:
        ключ - строка cluster_<номер_кластера>
        значение - самый встречающийся тег в данном кластере
        '''
        dominant_tag_in_clusters = {}

        for i in range(self.best_k):
            cluster_sents = c.clustered_sentences_with_tags[i][:, 0]
            cluster_tags = c.clustered_sentences_with_tags[i][:, 1]

            # достаем индексы предложений, которые входят в self.X_train
            cluster_train_indices = np.in1d(cluster_sents, self.X_train)
            cluster_tags = cluster_tags[cluster_train_indices]
            dominant_tag_in_clusters['cluster_' + str(i)] = \
                                    Counter(cluster_tags).most_common(1)[0][0]
        return dominant_tag_in_clusters

    def assign_major_cluster_label(self):
        '''
        создает нампай массив:
        первая колонка - исходный текст
        вторая - новый тег (по большинству из кластера)
        '''
        dct = self.create_dominant_tags_in_clusters()
        arr = np.zeros((len(self.corpus), 2), dtype='object')
        sents_added = arr[:, 0]  # ссылка на предложения (нулевая колонка в массиве arr)
        for i, sent_tag in enumerate(self.corpus):
            cur_sent = self.corpus[i]
            if cur_sent not in sents_added:
                cluster_num = self.cluster_labels[i]
                arr[i] = [self.corpus[i], dct['cluster_' + str(cluster_num)]]

        arr = arr[arr[:, 0] != 0]  # нули, потому что arr - матрица нулей
        assert(len(arr) == len(set(self.corpus)))
        return arr

    def save_numpy_object(self, filename: str):
        np.save(filename, self.assign_major_cluster_label())

    def save_embeddings(self):
        dct = self.create_dominant_tags_in_clusters()
        arr = np.zeros((len(self.corpus_embeddings), 2), dtype='object')
        for i, sent_tag in enumerate(self.corpus_embeddings):
            cluster_num = self.cluster_labels[i]
            arr[i] = [self.self_embeddings[i], dct['cluster_' + str(cluster_num)]]
        if DATASET_NAME == 'vk':
            np.save(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/emb_gmm_new_{DATASET_NAME}')
        else:
            np.save(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/emb_gmm_new_{DATASET_NAME}_{LABELING_OPTION}')
        return arr

    def create_tag_dict(self):
        '''
        сейчас используется только в self.plot_distro_of_clusters_in_single_tag
        для подсчета распределения того, какие кластеры попали в определенный тег
        '''
        res = {}
        for tag, cl_label in zip(self.tags, self.cluster_labels):
            res.setdefault(tag, []).append(cl_label) 
        return res

    def plot_distro_of_clusters_in_single_tag(self):
        '''
        подсчет распределения различных кластеров в одном теге
        '''
        n_axes = len(Counter(self.tags).keys())
        fig, axes = plt.subplots(nrows=n_axes, ncols=1, figsize=(n_axes * 4, n_axes * 5))# figsize=(20, n_axes * 10))
        distro = self.create_tag_dict()
        distro = list(distro.items())
        for i, ax in enumerate(axes):
            tag_clusters_counter = Counter(distro[i][1]).most_common()
            tag_clusters_counter = sorted(tag_clusters_counter, key=lambda x: x[0])
            tag_clusters_counter = dict(tag_clusters_counter)
            ax.set_title(f'Tag: {distro[i][0]}')
            x_labels = tag_clusters_counter.keys()
            ax.bar(tag_clusters_counter.keys(), tag_clusters_counter.values())
            ax.set_xticks(np.arange(0, self.best_k))
            ax.set_xticklabels(x_labels, rotation=90)
            plt.tight_layout()
        if DATASET_NAME == 'vk':
            plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/clusters_in_tags_{DATASET_NAME}', dpi=300)
        else:
            plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/clusters_in_tags_{DATASET_NAME}_{LABELING_OPTION}', dpi=300)

    def plot_distro_of_tags_in_single_cluster(self):
        '''
        подсчет распределения различных тегов в одном кластере
        '''
        fig, axes = plt.subplots(nrows=self.best_k, ncols=1, figsize=(40, self.best_k * 10))
        for i, ax in enumerate(axes):
            cluster_tags_counter = Counter(np.array(self.clustered_sentences_with_tags[i])[:, 1]).most_common()
            cluster_tags_counter = sorted(cluster_tags_counter, key=lambda x: x[0])[1:]
            cluster_tags_counter = dict(cluster_tags_counter)
            ax.set_title(f'Cluster # {i}')
            ax.set_xlabel('Tags')
            x_labels = cluster_tags_counter.keys()
            ax.bar(x_labels, cluster_tags_counter.values())
            ax.set_xticklabels(x_labels, rotation=90)
        fig.tight_layout()

        if DATASET_NAME == 'vk':
            plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/tags_in_clusters_{DATASET_NAME}', dpi=300)
        else:
            plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/tags_in_clusters_{DATASET_NAME}_{LABELING_OPTION}', dpi=300)

    def write_clustering(self, filename='file.txt'):
        res = {}
        sent_tag_array = self.assign_major_cluster_label()
        for sent, tag in sent_tag_array:
            res.setdefault(tag, []).append(sent) 
        with open(filename, 'w') as fh:
            for cluster in res:
                fh.write(f'Клаcтер # {cluster} #\n\n')
                for sent in res[cluster]:
                    fh.write(sent+'\n')

## Agglomerative Clustering + GMM

GMM sources

[Kmeans vs GMM](https://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html)

[Clustering with Gaussian Mixture Models](https://pythonmachinelearning.pro/clustering-with-gaussian-mixture-models/)

[Evaluation](https://stackoverflow.com/questions/59827051/evaluating-gaussian-mixture-model-using-a-score-metric)

In [ ]:
c = Clustering(corpus, tags, max_len=MAX_LEN)\
            .tokenize()\
            .create_data_loader(16)\
            .make_embeddings()\
            .save_embeddings()


Time taken: 12 minutes and 35 seconds


#### Собственно кластеризация

In [ ]:
# Agglomerative
kwargs_hac = {
    'affinity': 'euclidean',
    'linkage': 'ward',
}

# Gaussian
kwargs_gmm = {
    'covariance_type': 'full',
    'random_state': 42,
    'tol': 5e-3
}

# KMeans
kwargs_kmeans = {
    'tol': 1e-3,
    'max_iter': 200
}


# c.cluster(KMeans, 10, 11)
# c.cluster(AgglomerativeClustering, 5, 25, **kwargs_hac)
# c.cluster(GaussianMixture, 25, 50, **kwargs_gmm)
c.cluster(GaussianMixture, 20, 20, **kwargs_gmm)

Running with k = 20
	Done for k = 20
Time taken: 1 minutes and 55 seconds
Clustering with the best k = 20
#### DONE ####


In [ ]:
def plot_distro_of_clusters_in_single_tag():
    '''
    подсчет распределения различных кластеров в одном теге
    '''
    n_axes = len(Counter(c.tags).keys())
    fig, axes = plt.subplots(nrows=n_axes, ncols=1, figsize=(n_axes * 4, n_axes * 5))
    distro = c.create_tag_dict()
    distro = list(distro.items())
    for i, ax in enumerate(axes):
        tag_clusters_counter = Counter(distro[i][1]).most_common()
        tag_clusters_counter = sorted(tag_clusters_counter, key=lambda x: x[0])
        tag_clusters_counter = dict(tag_clusters_counter)
        x_labels = tag_clusters_counter.keys()
        ax.bar(tag_clusters_counter.keys(), tag_clusters_counter.values(), edgecolor='k')
        ax.set_xticks(np.arange(0, c.best_k))
        ax.set_xticklabels(x_labels)
        ax.set_xlabel('Cluster number', fontsize=16)
        ax.set_ylabel('Number of examples', fontsize=16)
        ax.set_title(f'Tag: {distro[i][0]}', fontsize=18)
        ax.tick_params(labelsize=14)
        plt.tight_layout()
    if DATASET_NAME == 'vk':
        plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/clusters_in_tags_{DATASET_NAME}', dpi=300)
    else:
        plt.savefig(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/clusters_in_tags_{DATASET_NAME}_{LABELING_OPTION}', dpi=300)

Сохранить нампай объект, записать тексты в различных кластерах

In [ ]:
# if DATASET_NAME == 'vk':
#     c.save_numpy_object(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_new_{DATASET_NAME}')
# else:
#     c.save_numpy_object(f'/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_new_{DATASET_NAME}_{LABELING_OPTION}')

# # c.write_clustering('/content/gdrive/MyDrive/RealEstate/sem_4/Final/gmm_clustered_classes.txt')

# Testing clustering

In [ ]:
from sklearn.metrics import f1_score

preds = c.assign_major_cluster_label()[:, 1][c.test_idxs]
f1_score(preds, c.y_test, average='micro')

0.274105599148823

In [ ]:
# проверим, что индексы корректные
x = c.test_idxs[1]
c.corpus[x], c.assign_major_cluster_label()[:, 0][x]

('Кто нибудь еще ходит в библиотеки В 132 ю привезли новые книжки для детей',
 'Кто нибудь еще ходит в библиотеки В 132 ю привезли новые книжки для детей')

# "Точность" кластеризации на тестовой выборке

In [ ]:
test_clustering_tags = c.assign_major_cluster_label()[:, 1][c.test_idxs]
test_initial_tags = np.asarray(all_tags)[c.test_idxs]

In [ ]:
test_initial_tags[:50]

array([list(['Мероприятие']), list(['Новость']), list(['Новость']),
       list(['Новость']), list(['Мероприятие', 'Дети']),
       list(['#лучшедома']), list(['Вопрос']),
       list(['#лучшедома', 'Вопрос']), list(['Безобразие', 'Здоровье']),
       list(['Настроение']), list(['Безобразие', 'Пешеходу']),
       list(['Животные']), list(['Мероприятие']), list(['#лучшедома']),
       list(['Работа']), list(['Услуги']), list(['Новость', 'Дети']),
       list(['Образование']), list(['Новость']), list(['#лучшедома']),
       list(['Вопрос']), list(['Новость']), list(['Мероприятие']),
       list(['#лучшедома']), list(['Безобразие']), list(['#лучшедома']),
       list(['Животные']), list(['Образование']), list(['Услуги']),
       list(['Дети', 'Образование']), list(['Предупреждение']),
       list(['Вопрос']), list(['Новость']),
       list(['Настроение', 'Скидки']), list(['Дети']),
       list(['#лучшедома']), list(['Благотворительность', 'Приму в дар']),
       list(['Водителю', 'Происше

In [ ]:
3039 in c.test_idxs, all_tags[3039]

(False, ['Новость'])

In [ ]:
count = 0
for tag, tag_list in zip(test_clustering_tags, test_initial_tags):
    if tag in tag_list:
        count += 1

In [ ]:
count / len(test_clustering_tags)

0.3104136188322915